# Table of Contents
## 0. [Imports, data loading and initial checking](#DL)
## 1. [Features properties](#FP)
## 2. [Features enginering](#FE)
## 3. [Model hipothesis and discussion](#MH)
## 4. [Model building](#MB)
## 5. [Model selection](#MS)
## 6. [Conclussions](#C)
## 7. [Submission](#S)

In [ ]:
import os
import re
import sys 
sys.path.insert(0, '../src/visualization/') # add the path which contains visualize
sys.path.insert(0, '../src/data/') # add the path which containing visualize
sys.path.insert(0, '../src/features/') # add the path which contains visualize

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import visualize as vsl  # own visualize.py file containing every visualization scripts
import fill_nulls as fln  # own fill_nulls.py file containing scripts to fill nan
import build_features as bf  # own build_features.py containing the script to generate features dataset 
from titanic_submission import submit_result  # own titanic_submission.py script to automatize submissions

from scipy.stats import kstest, kurtosis, shapiro, skew, boxcox
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from statsmodels.discrete.discrete_model import Logit
from sklearn.preprocessing import StandardScaler

In [ ]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = [9,6]

# 0. Imports, data loading and initial checking <a class="anchor" id="DL"></a>

In [ ]:
train_data = pd.read_csv('../data/raw/train.csv')
train_data.head()

In [ ]:
train_data.info()

We can see that Age, Embarked and Cabin contain null values, so, we will keep it in mind (in order to fill them) if we consider that some of these features is relevant after the properties overview.

# 1. Features properties <a class="anchor" id="FP"></a>

In this section we are going to study the features properties in order to two main aspects:

1. Marginal Distribution to know how a feature acts marginally 

2. Joint distribution with Survived to know which features are significantly related with the survivors

The methodology is basicly take an overview with a pie chart (in the case of cathegorical features) or an histogram (in the case of numerical values), then we will plot the joint distribution with a stacked bars diagram (in the case of cathegorical features) or an histogram hued by survivor (in the case of numerical features).

## 1.1 PassengerId

This feature seems to be a unique identifier for every passenger, but we do not need it because it matches with our dataframe indexes. It will be removed.

In [ ]:
train_data.drop('PassengerId', axis=1, inplace=True)

## 1.2 Pclass

### MARGINAL DISTRIBUTION

In [ ]:
vsl.build_pie_chart(train_data['Pclass'])

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
vsl.build_stacked_bars_chart(train_data['Pclass'], train_data['Survived'])

From the above graphic we can see a relation between Survived and Pclass, as we can see, when Pclass decreases survivors proportion increases, so, it could be a feature in the model.

## 1.3 Sex

### MARGINAL DISTRIBUTION

In [ ]:
vsl.build_pie_chart(train_data['Pclass'])

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
vsl.build_stacked_bars_chart(train_data['Sex'], train_data['Survived'])

We can see that females represent aprroximately the thitd part of the passage, but, they have a significant higher probability to survive than males, so, it could be a feature in the model due to the strong relation.

## 1.4 Age

### MARGINAL DISTRIBUTION

In [ ]:
plt.hist(train_data['Age'])
plt.title('Marginal distribution of Age')
plt.show()

From the above histogram we can conclude that a great majority of the passage is between 18 and 30 years approximately. It seems a distribution skewed to the right and it is not any well-known distribution. Specially, The normality assumption it is not justified as we can see in the Kolmogorov-Smirnov pvalue.

In [ ]:
kstest(train_data['Age'].dropna(), 'norm')

In spite of non normality, we can assume that the distribution is approximately symmetric as we can see in kurtosis and skew coefficients.

In [ ]:
print('kurtosis:', kurtosis(train_data['Age'].dropna()), 'skew:', skew(train_data['Age'].dropna()))

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
sns.histplot(data=train_data, x='Age', hue='Survived')

We can notice than childs have the highest chance to survive, and in the center of the distribution (young and middle-age persons) have approximately the same probability to survive or not. For the oldest people we can see that, in overall, nobody survived. 

That feature seems useful to predict survivors,so, we will include it in the model, but first, we have to implement a method to fill the null values.

## 1.5 SibSp

### MARGINAL DISTRIBUTION

In [ ]:
vsl.build_pie_chart(train_data['SibSp'])

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
vsl.build_stacked_bars_chart(train_data['SibSp'], train_data['Survived'])

We can see that the probability of survive decreases when SibSp increases (in overall), this feature could be included in the model.

## 1.6 Parch

### MARGINAL DISTRIBUTION

In [ ]:
vsl.build_pie_chart(train_data['Parch'])

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
vsl.build_stacked_bars_chart(train_data['Parch'], train_data['Survived'])

From the joint distribution we can see a (slight) relation between the Parch and Survived, where Parch increases the probability of survive decreases.

*Comment*: In overall, that feature has the same relation with survived as SibSp, so, it could be a good idea merge both features in a unique feature to prevent correlation between features in the future model.

## 1.7 Fare

### MARGINAL DISTRIBUTION

In [ ]:
sns.histplot(data=train_data, x='Fare', bins=15)

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
sns.histplot(data=train_data, x='Fare', hue='Survived', bins=15)

We can see that the probability of survive increases when Fare increases, so, it could be a feature in the model, but, probably, its correlated with Pclass, we have to take care if we include both features to prevent high correlations between model features.

## 1.8 Embarked

### MARGINAL DISTRIBUTION

In [ ]:
vsl.build_pie_chart(train_data['Embarked'])

### JOINT DISTRIBUTION WITH SURVIVED

In [ ]:
vsl.build_stacked_bars_chart(train_data['Embarked'], train_data['Survived'])

We can see a slight relation between the place where passenger embarked and the probability of survive, but it does not seem significant. We will test different models including or not this feature and we will se if it is relevant or not.

In addition, it has null values as we can see below.

In [ ]:
train_data['Embarked'].isnull().sum()

Due to the small number of null values we will replace it with the mode.

In [ ]:
mode_embarked = train_data['Embarked'].mode()[0]

In [ ]:
train_data.loc[train_data['Embarked'].isnull(), 'Embarked'] = mode_embarked
train_data['Embarked'].isnull().sum()  # check if the null values are filled

## 1.9 Name

In [ ]:
train_data.head(5)

In a simplistic approach, names do not seem a reason to survive a disaster, but, if we remind that we need to fill Age null values, definetly, the salutations before the surname can be very useful to locate persons in an age band. We will develop these ideas later.

## 1.10 Cabin

It is reasonable to think that cabins distribution could be a reason to survive in order to some natural factors as the distance to the nearest safeboats or the initial hole in the boat, so, we are going to check if this feature has a relation with survived.

In [ ]:
vsl.build_stacked_bars_chart(train_data['Cabin'].fillna('Missing').apply(lambda x: x[:1]),
                             train_data['Survived'].loc[train_data['Cabin'].index])

We can see that the survive probability conditioned by the cabin first letter is different for every Cabin value, Cabins B, D and E have a higher proportion of suvivors than G, C, A. In addition every passengers in Cabin T died. It seems to be a relations and we could include that feature in the model.

Note that we have filled the null values with the Cabin label 'Missing', it could be reasonable, as we can see above, beacause a great part of the passengers who traveled in that Cabin died, so, it is natural that their Cabin is unknown. 

Lets check too the relation between Cabin and some other variables.

In [ ]:
vsl.build_stacked_bars_chart(train_data['Cabin'].fillna('Missing').apply(lambda x: x[:1]),
                             train_data['Pclass'].loc[train_data['Cabin'].index])

We notice that Cabins could be assigned by class, A, B, and C is assigned for the first class passengers, the others have mixed classes. So, we have to take care about correlation between both features.

# 2. Features enginering <a class="anchor" id="FE"></a>

## 2.1 Name. A way to fill Age null values.

In this section, as mentioned previously, we are going to extract passengers salutations with the goal to extract their titles, in addition, it could be a good stratification feature in order to infer an age band for passengers which would allow us to fill Age null values.

Let's take a look of the names format in the dataset.

In [ ]:
train_data.head()

The format seems to be 'name, salutation. surname' so, the appropiate tool to get the salutations will be  a regex rule runned over every Name to keep the string inmediately before the points.

In [ ]:
train_data['Title'] = train_data.Name.str.extract('([A-Za-z]+)\.', expand=False) 
train_data['Title'].value_counts(normalize=True)  # overview of saludations

We can see that there common salutations such as Mr o Miss, but, we have find multiple salutations which are rare such as Countess or Jonkheer. Seems reasonable to make groups in order to the most common titles and mke and additional group containin rare titles:

1. Mr and Mrs: man and women who are married
2. Miss: women not married
2. Master: an special title for minors. 
3. Rare: those which are not included above

In [ ]:
train_data['Title'] = train_data['Title'].apply(lambda x: 
                          1 if x in ['Mr', 'Mrs'] else 
                          (2 if x == 'Miss' else 
                           (3 if x == 'Master' else 4)))

group_means = []

for age_group in train_data['Title'].unique().tolist():
    
    group_means.append(train_data[train_data['Title'].isin([age_group])]['Age'].mean())
    
means_dict = dict(zip(train_data['Title'].unique().tolist(), group_means)) # dict to use with replace

as we can see our stratification seems reasonable, every group has a different age mean.

In [ ]:
pd.DataFrame.from_dict(means_dict, orient='index', columns=['mean']).transpose()

Finally we can fill the values.

In [ ]:
ages_to_fill = train_data[train_data['Age'].isnull()]['Title'].replace(means_dict)
ages_to_fill = ages_to_fill.apply(int) # transform age into int type
train_data.loc[ages_to_fill.index, 'Age'] = ages_to_fill

train_data['Age'].isnull().sum()  # we can check that there is no null values now

Before leave this section it could a good idea check the joint distribution of title and Survived, as we said above, Age is related with Survived, so, it is reasonable that our new feature, Title, has inherited that relation, because salutations are based in the age, experience and knowledge.

In [ ]:
vsl.build_stacked_bars_chart(train_data['Title'], train_data['Survived'])

As expected, the relation is inherited, and probably, it could be a good feature rather than simply Age, we will see in the model selection.

## 2.2 Family features

As we concluded above, SibSp and Parch can duplicate information about a passenger family status in the model, because they showed a similar relation with Survived. It could be a good idea to summarize both features in only one boolean feature that we will call Alone, which values are 1 if the passengers travles alone and 0 if the passenger travels with some family member/s.

In [ ]:
train_data['Alone'] = (train_data['SibSp'] + train_data['Parch'] + 1).apply(lambda x: 0 if x!=1 else 1)

As always, now we check the relations with Survived

In [ ]:
vsl.build_stacked_bars_chart(train_data['Alone'], train_data['Survived'])

And we can see a slight relation between both variables, we will study the feature in the model selection.

## 2.3 Age Features

As an intuition, I propose to create a boolean feature about the elder persons. It reasonable to think that persons who are very old, are more delicated than young persons, so, we will define the threshold to consider someone old in 65 years old, and we will check the relation with Survived.

In [ ]:
train_data['Old'] = train_data['Age'].apply(lambda x: 1 if x>65 else 0)

In [ ]:
vsl.build_stacked_bars_chart(train_data['Old'], train_data['Survived'])

It seems a strong relation, if some passenger is more than 65, its probability to survive is very low, in the other hand, persons who are not old have approximately the same probability of survive or not, and we could not reject the possibility that the hipothesis is true. We will check it in the model selection.

# 3. Model hipothesis and discussion <a class="anchor" id="MH"></a>

Now we are going to analyse the features and the target variable in order to propose a set of models which can predict which passengers will survive.

* Target feature (to predict): boolean
* Avaible features: boolean, cathegorical, numerical.

In addition, it is highly posible that some variables cumulate information from others, for example, seems reasonable that the highest fares are concentrated in the first Pclass, as we can see below:

In [ ]:
sns.histplot(data=train_data, x='Fare', hue='Pclass', bins=10, palette='summer')

As a conclussion from the above exploratory analysis, it seems clear that we have to solve a classification problem, so, the classical techniques used in this kind of problems are: Logistic Regression, Decission Trees and his variant Random Forest, Support Vector Classifier, linear classifiers (SVG), Naive Bayes Classifiers...

At the beginning, we will try to fit the most simple and classical models such as Logistic Regression and Decission Trees. If it do not work as expected, we will add more complexity.

# 4. Model building <a class="anchor" id="MB"></a>

In this section we are going to prepare the data in order to apply models over a prepared features dataset, we will create all necessary dummy features and copy the original dataset columns in order to prevent undesiderable modifications.

## 4.1 Features creation

In [ ]:
#sex featrue
Isfemale = pd.get_dummies(train_data['Sex'])['female']

# extra features from Age, not very useful
# Isold = train_data['Old'].copy()  

# SibSp features 
Isalone = train_data['Alone'].copy()
SibSp = train_data['SibSp'].copy()
Parch = train_data['Parch'].copy()

# class features
C1 = pd.get_dummies(train_data['Pclass'])[1]
C2 = pd.get_dummies(train_data['Pclass'])[2]
Pclass = train_data['Pclass']

# numeric features
scaler = StandardScaler()

Age = train_data['Age'].copy()
Age = pd.Series(scaler.fit_transform(np.array(Age).reshape(-1,1))[:, 0])
#Age (bins)
Age_bins = pd.cut(train_data['Age'], bins=10, labels=range(10))


Fare = train_data['Fare'].copy()
Fare = pd.Series(scaler.fit_transform(np.array(Fare).reshape(-1,1))[:, 0])

# Fare bins
Fare_bins = pd.cut(train_data['Fare'], bins=10, labels=range(10))

# embarked features
Isq = pd.get_dummies(train_data['Embarked'])['Q']
Iss = pd.get_dummies(train_data['Embarked'])['S']

# title features
IsMr = train_data['Name'].apply(lambda x: 1 if 'Mr' in x else 0)
IsMrs = train_data['Name'].apply(lambda x: 1 if 'Mrs' in x else 0)
Isminor = pd.get_dummies(train_data['Title'])[3]
Ismiss = pd.get_dummies(train_data['Title'])[2]
Israre = pd.get_dummies(train_data['Title'])[4]

#Cabin features
Cabin = pd.get_dummies(train_data['Cabin'].fillna('Missing').apply(lambda x: x[:1]))
Cabin['M'] = Cabin['M'] + Cabin['T']  # add T Cabin to M
Cabin.drop('T', axis=1, inplace=True)  # remove Cabin T

X = pd.concat([Isfemale, Isalone, C1,
           C2, Age, Fare, Isq, Iss, IsMr, IsMrs,
           Isminor, Ismiss, Israre, SibSp, Parch,
               Pclass, Age_bins, Fare_bins, Cabin], axis=1, ignore_index=True)

feature_names = ['Isfemale', 'Isalone', 'C1',
           'C2', 'Age', 'Fare', 'Isq', 'Iss', 'Ismr', 'Ismrs',
           'Isminor', 'Ismiss', 'Israre', 'SibSp', 'Parch', 'Pclass',
                 'Age_bins', 'Fare_bins', 'CabinA','CabinB',
                 'CabinC', 'CabinD', 'CabinE',	'CabinF', 'CabinG', 'CabinM']

X.columns = feature_names

y = train_data['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## 4.2 Running basic models

Now we split model data in train and test samples, and fit, as we said, a Logistic Regression model and a Random Forest model.

## DECISSION TREES

In [ ]:
def run_decission_tree(features):
    
    DT = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=13,
                                min_samples_split=4)

    DT.fit(X_train[features], y_train)

    scores = cross_val_score(DT, X_train, y_train, cv=10)
    print('average score:', np.mean(scores))

    plt.bar(range(len(scores)), scores)
    plt.xlabel('folds')
    plt.ylabel('score obtained')
    plt.title('Cross validation result')
    plt.hlines(np.mean(scores), xmin=0, xmax=9, linestyles='dashed', colors='black')
    plt.legend(['mean', 'result CV'])
    plt.show()

    print(pd.DataFrame(zip(features, DT.feature_importances_),columns=['feature', 'importance']))
    print('accuracy_training:', DT.score(X_train[features], y_train))
    DT.fit(X, y)

    return DT, features

In [ ]:
DT, features_DT = run_decission_tree(set(feature_names)-set(['C1', 'C2', 'Fare', 'Age']))

## RANDOM FOREST

In [ ]:
def run_random_forest(features):
    
    RFR = RandomForestClassifier(bootstrap=True, criterion='gini',
                                max_features=0.25, min_samples_leaf=4,
                                min_samples_split=3, n_estimators=100, random_state=0)

    RFR.fit(X_train[features], y_train)

    scores = cross_val_score(RFR, X_train, y_train, cv=10)
    print('average score:', np.mean(scores))

    plt.bar(range(len(scores)), scores)
    plt.xlabel('folds')
    plt.ylabel('score obtained')
    plt.title('Cross validation result')
    plt.hlines(np.mean(scores), xmin=0, xmax=9, linestyles='dashed', colors='black')
    plt.legend(['mean', 'result CV'])
    plt.show()

    print(pd.DataFrame(zip(features, RFR.feature_importances_),columns=['feature', 'importance']))

    print('accuracy_training:', RFR.score(X_train[features], y_train))
    RFR.fit(X, y)
    return RFR, features

In [ ]:
RFR, features_RFR = run_random_forest(set(feature_names)-set(['C1', 'C2',
                                                              'Age', 'Fare', 'Isfemale',
                                                              'CabinG', 'Isalone']))

## LOGISTIC REGRESSION

In [ ]:
def run_logit(features):
    
    logit = LogisticRegression(max_iter=1000)
    logit.fit(X_train[features], y_train)
    scores = cross_val_score(logit, X_train, y_train, cv=10)

    print('average score:', np.mean(scores))

    plt.bar(range(len(scores)), scores)
    plt.xlabel('folds')
    plt.ylabel('score obtained')
    plt.title('Cross validation result')
    plt.hlines(np.mean(scores), xmin=0, xmax=9, linestyles='dashed', colors='black')
    plt.legend(['mean', 'result CV'])
    plt.show()
    
    LR = Logit(y_train, X_train[features])
    result = LR.fit()
    print(result.summary2())
    
    print('accuracy_training:', logit.score(X_train[features], y_train))
    logit.fit(X, y)
    
    return logit, features

In [ ]:
LR, features_LR = run_logit(set(feature_names)-set(['C1', 'C2', 'Age', 'Fare',
                                                    ]))

# 5. Model selection <a class="anchor" id="MS"></a>

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In the previous section, both models performed very similar, now, we will discuss some other alternatives.

## SGDC

In [ ]:
GDC = SGDClassifier(random_state=0, alpha=0.01, eta0=0.01, fit_intercept=True,
             l1_ratio=0.5, learning_rate='constant', loss='hinge',
             penalty='elasticnet', power_t=50.0)

GDC.fit(X, y)
scores = cross_val_score(GDC, X, y, cv=10)
np.mean(scores)

## EXTRATREESCLASSIFICATOR

In [ ]:
ETC = ExtraTreesClassifier(random_state=0, bootstrap=True, criterion='entropy',
                           max_features=0.35000000000000003, min_samples_leaf=12,
                           min_samples_split=18, n_estimators=100)
ETC_features = set(feature_names)-set(['C1', 'C2', 'Age', 'Fare'])
ETC.fit(X[ETC_features], y)
scores = cross_val_score(ETC, X[ETC_features], y, cv=10)
np.mean(scores)

## MULTILAYER PERCEPTRON

In [ ]:
MLP = MLPClassifier(alpha=0.001, learning_rate_init=0.001, max_iter=1000)
MLP.fit(X_train, y_train)
print(MLP.score(X_test, y_test))
scores = cross_val_score(MLP, X, y, cv=10)
np.mean(scores)

## XGB classifier

In [ ]:
XGB = XGBClassifier(learning_rate=0.01, max_depth=5,
              min_child_weight=2, n_estimators=100, nthread=1, subsample=0.6500000000000001)
XGB.fit(X, y)
np.mean(cross_val_score(XGB, X, y, cv=10))

# 6. Conclussions <a class="anchor" id="C"></a>

In [ ]:
break

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
tpot = TPOTClassifier(generations=10, population_size=40, verbosity=1, scoring='accuracy', cv=cv)
tpot_features = set(feature_names)-set(['C1', 'C2','Age', 'Fare', 'Isfemale',
                                                              'CabinG', 'Isalone'])
tpot.fit(X[tpot_features], y)

# 7. Submission <a class="anchor" id="S"></a>

For the submission we will use the scripts, but the process is exactly the same as the above cells.

In [ ]:
test = pd.read_csv('../data/raw/test.csv')

fln.fill_age_values(test)
fln.fill_embarked(test)
fln.fill_fare_value(test)

sub_features, _ = bf.build_features(test)

In [ ]:
submit_result(sub_features, tpot, list(tpot_features))